In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import confusion_matrix, precision_recall_curve

torch.manual_seed(420)

import warnings
warnings.filterwarnings("ignore")

# Postavka problema: Opstanak novih igrača u NBA ligi

Jedan od čestih problema menadžera timova u NBA ligi je procena da li je igrač koji je upravo došao u ligu ima potencijala da u njoj igra na duže staze. 
Menadžeri se pored mišljenja trenera i ostalih igrača vrlo često oslanjaju na analizu statistike igrača.
U ovom radu predlažemo model koji će na osnovu statistike prve sezone igrača u NBA generisati predlog da li je vredno zadržati igrača u timu.

Treniranje i evaluacija vršena je na datasetu [5 Year Survival of NBA Rookies from 1980-2015](https://www.kaggle.com/datasets/mamadoudiallo/5-year-survival-of-nba-rookies-from-19802015) u kojem je računata isplativost zadržavanja igrača u timu ako on i dalje ima ugovor nakon 5 godina po dolasku u ligu.
Dataset sadrži sve draftovane igrače u periodu od 1980. do 2015, njihovu statistiku u prvoj sezoni, kao i to da li su ostali u ligi nakon 5 godina.

# Istraživanje podataka i predprocesiranje

U nastavku su podaci učitani u *dataframe* i urađeno je osnovno predprocesiranje kako bi se smanjila dimenzionalnost problema.
Svaka od odluka pri filtriranju baze biće objašnjena neposredno pre ili posle odgovarajućeg bloka koda.

In [ ]:
# Load .csv file into pandas dataframe
df = pd.read_csv("data/rookie_df.csv", low_memory=False)
df

In [ ]:
# Print all column names
df.columns

### Odabir relevantnih atributa

U datasetu postoji 22 atributa za svakiog igraca, medju kojima postoje neki koji su očigledno korelisani.
Uzmimo za primer slobodna bacanja: za svakog igrača je vođena statistika koliko bacanja je igrač šutira (`FGA`), koliko ih je pogodio (`FGM`), kao i procenat uspešnih bacanja (`FG%`).
Očigledno je da je procenat moguće nedvosmisleno iračunati iz prva dva podatka, tako da je on otklonjen.
Odluka da se zadže dva broja umesto procenta doneta je kako bi model mogao da uračuna situacije u kojima je igrač imao mali broj pokušaja i samim time procenat ne prenosi adekvatno njegovu stvarnu efikasnost.

U nastavku je zadržano 16 atributa i otklonjeni su svi igrači sa nedefinisanim poljima.

| Atribut | Opis                                        |
|---------|---------------------------------------------|
| GP      | Broj odigranih utakmica                     |
| MIN     | Prosečan broj minuta u igri                 |
| PTS     | Prosečan broj postignutih poena             |
| FGM     | Prosečan broj postignutih šuteva iz igre    |
| FGA     | Prosečan broj pokušanih šuteva iz igre      |
| 3P Made | Prosečan broj postignutih šuteva za 3 poena |
| 3PA     | Prosečan broj pokušanih šuteva za 3 poena   |
| FTM     | Prosečan broj pogođenih slobodnih bacanja   |
| FTA     | Prosečan broj pokušanih slobodnih bacanja   |
| OREB    | Prosečan broj skokova u napadu              |
| DREB    | Prosečan broj skokova u odbrani             |
| AST     | Prosečan broj asistencija                   |
| STL     | Prosečan broj ukradenih lopti               |
| BLK     | Prosečan broj blokiranih šuteva             |
| TOV     | Prosečan broj izgubljenih lopti             |
| EFF     | Prosečan skor efikasnosti igrača            |

In [ ]:
# Choose valuable features
clean_df = df[['GP', 'MIN', 'PTS', 'FGM', 'FGA',
       '3P Made', '3PA', 'FTM', 'FTA', 'OREB', 'DREB',
       'AST', 'STL', 'BLK', 'TOV', 'EFF', 'target']].copy()
clean_df.dropna(inplace=True)

In [ ]:
# Describe features
clean_df.describe()

In [ ]:
# Calculate percentage of True datapoints
value_cnts = clean_df["target"].value_counts()
print(f"Dataset sadrži {len(clean_df)} igrača od kojih je {value_cnts[1] / len(clean_df) * 100:0.2f}% ostalo u ligi nakon 5 godina.")

### Podela na trening i test

Dataset je podeljen na trening i test skup, od čega trening set sadrži 70% podataka.
Korištena je fukncija `train_test_split` koja nasumično promeni mesta podataka u tabeli pre nego što uradi podelu na dva seta. 

In [ ]:
from sklearn.model_selection import train_test_split

# Do train/test split
train_df, valid_df = train_test_split(clean_df, train_size=.7, random_state=420)

### Normalizacija dataseta

Pošto se svi atributi u datasetu ne nalaze u istim opsezima, izvršena je normalizacija podataka.
Za ovu svrhu upotrebljena je `StandardScaler` klasa, ima mogućnost fitovanja parametara normalizacije na jednom skupu, i odvojenu primenu iste.
Parametri normalizacije su fitovani na trening setu, a onda primenjeni i na treningu i na testu.
Na ovaj način nije došlo do curenja informacija iz testa u trening.

In [ ]:
from sklearn.preprocessing import StandardScaler

input_columns = clean_df.columns[:-1]

# Fit scaler
scaler = StandardScaler()
scaler.fit(train_df[input_columns].values)

# Normalize train data
norm_train_np = scaler.transform(train_df[input_columns].values)
norm_train_df = pd.DataFrame(norm_train_np, columns=input_columns)
norm_train_df["target"] = train_df["target"].values

# Normalize validation data
norm_valid_np = scaler.transform(valid_df[input_columns])
norm_valid_df = pd.DataFrame(norm_valid_np, columns=input_columns)
norm_valid_df["target"] = valid_df["target"].values

Ovime je završeno predprocesiranje podataka, i oni su spremni za treniranje modela.

# Definisanje modela

Radi predikcije dugovečnosti igrača na osnovu statistike u prvoj godini implementirana je poptpuno povezana neuronska mreža.
Mrežu je moguće inicilaizovati sa proizvoljnim brojem skrivenih slojeva, a postoji tačno jedan izlazni neuron.
Izlaz modela predstavlja njegovo poverenje da će igrač opstati u ligi, u rasponu [0, 1].

Svi delovi generisanja modela, treninga i evaluacije dekomponovani su u odgovarajuće funkcije i klase radi jednostavnijeg eksperimentisanja kasnije.

### Klasa za učitavanje podataka

Tokom treninga je poželjno da pri tokom svake epohe treninga, podaci budu prosleđeni modelu u različitim redosledima.
`pytorch` obezbeđuje ovu funkcionalnost, kao i jednostavno čitanje podataka po *batch*-evima iteracijom kroz klasu `Dataloader`.
Ova klasa pri svojoj inicijalizaciji zahteva objekat koji nasleđuje klasu `Dataset`.

`Dataset` je klasa koja sadrži sve podatke treninga/testa i potrebno je u njoj implementirati metod `__getitem__` koji vraća ulazne podatke kao i očekivanu vrednost za model, na osnovu indeksa u bazi.
U nastavku je implementirana klasa `NBA_Dataset`, koja obezbeđuje ovu funkcionalnost za odgovarajući dataset.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class NBA_Dataset(Dataset):
    """Class for holding data NBA rookie statistics, and if they remained in the league after 5 year ."""

    def __init__(self, df: pd.DataFrame) -> None:
        """Initialize torch Dataset based on pandas Dataframe."""
        super(NBA_Dataset, self).__init__()

        # Take all except the last column as inputs
        self.inputs = df.values[:, :-1].astype(float)

        # Cast Targets column to numpy
        self.targets = df["target"].values.astype(int)

        # Save dataframe as part of the class
        self.df = df

        # Define dataset length
        self.len = len(df)

    def __getitem__(self, index) -> dict:
        """Return dict with information of datapoint at `index`"""
        # Get input
        input_row = torch.Tensor(self.inputs[index, :])

        # Get output
        target = torch.Tensor([self.targets[index]])

        return {
            "Inputs": input_row,
            "Targets": target,
        }

    def __len__(self):
        """Get dataset length."""
        return self.len

Nakon inicijalizacije `Dataset` klase, ona je *wrap*-ovana u `Dataloader` klasu.
U okviru ove klase su definisane veličina *batch*-a, broj radnih jedinica, kao i opcija da li redosled podataka treba randomizovati pred svaku epohu.

In [ ]:
train_dataset = NBA_Dataset(norm_train_df)

train_parameters = {
    "batch_size": 64,
    "shuffle": True,
    "num_workers": 1,
}

training_loader = DataLoader(train_dataset, **train_parameters)

# Validation dataset
valid_dataset = NBA_Dataset(norm_valid_df)

valid_parameters = {
    "batch_size": 64,
    "shuffle": False,
    "num_workers": 1,
}

validation_loader = DataLoader(valid_dataset, **valid_parameters)

### Arhitektura modela

Model je implementiran u klasi `NBA_Survival_Predictor`.
Mrežu je moguće inicijalizovati sa proizvoljnim brojem skrivenih slojeva, čije se veličine prosleđuju konstruktoru u vidu liste.
Model je takođe moguće inicijalizovati sa proizvoljnom aktivacionom funkciojom, a ako ona nije prosleđena, aktivaciona fukcija će biti hiperbolički tangens.

U okviru `forward` funkcije implementiran je prolazak kroz model.
Podaci se provlače kroz sve inicijalizovane objekte u listi slojeva.
Objekti su naizmenično potpuno povezani slojevi i aktivacione funkcije, a umesto poslednje aktivacione funkcije primenjen sigmoid.
Sigmoidna funkcija je upotrebljena kao standardna aktivaciona funkcija izlaznog sloja, prilikom klasifikacionih problema.

In [ ]:
import torch.nn as nn

class NBA_Survival_Predictor(nn.Module):
    """Class for estimating chance of NBA rookie surviving in the league."""

    def __init__(self, layer_sizes: list, activation_function: nn.Module = None) -> None:
        """Initialize fully-connected model based on input parameters."""
        super(NBA_Survival_Predictor, self).__init__()

        # Check if input and output layers are passed
        assert len(layer_sizes) >= 2

        # Check if activation function is not passed
        if activation_function is None:
            activation_function = nn.ReLU()

        # Define activation function
        self.activation = activation_function

        # Initialize list of layers
        layer_list = []

        # Initialize each layer and activation function
        for in_size, out_size in zip(layer_sizes[:-1], layer_sizes[1:]):
            layer_list.append(nn.Linear(in_size, out_size))
            layer_list.append(nn.Tanh())

        # Cast layer list to nn.Module
        self.fc_layers = nn.ModuleList(layer_list)

    def forward(self, X) -> torch.Tensor:
        """Define network behavior when called on data."""
        # Pass data trough fully-connected layers
        # Skip last activation function
        for layer in self.fc_layers[:-1]:
            X = layer(X)
        
        # Calculate output probability as sigmoid of output
        output_probability = torch.sigmoid(X)

        return output_probability

### Definisanje treninga i evaluacije

U nastavku se nalazi implementacija funkcije za treniranje i evaluaciju modela.
Funkciji od ulaznih parametara očekuje model, optimizator i DICTIONARY trenaznih objekata.
DICTIONARY u sebi sadrži 4 polja, *loss* funkciju, *dataloader* za trening i validaciju, kao i broj epoha za trening.

U okviru funkcije, model je treniran po *batch*-evima, a zatim evaluiran.
Na kraju svake epohe, sačuvani su vrednost *loss*-a i tačnost modela.
Tačnost je odabrana kao glavna metrika performanse modela zato što je *dataset* balansiran.

In [ ]:
CLASS_THR = .5
def train_network(model: NBA_Survival_Predictor, optimizer: torch.optim.Optimizer, training_objects: dict) -> tuple:
    """Function trains and evaluates defined model using given optimizer on passed training objects.

    Returned values of the function are trained model, and a dictionary with training and evaluation
    metrics.

    Args:
        model (NBA_Survival_Predictor): Model for training.
        optimizer (torch.optim.Optimizer): Model optimizer.
        training_objects (dict): Dictionary containing loss function, number of epochs, and
                                 training and validation loaders.

    Returns:
        tuple: Passed model after training, and dictionary containing training and validation metrics.
    """

    # Unpack dictionaries
    loss_function = training_objects["Loss Function"]
    training_loader = training_objects["Train Dataloader"]
    validation_loader = training_objects["Valid Dataloader"]
    num_of_epochs = training_objects["Epochs"]

    # Initialize lists for logging training and validation metrics
    training_loss = []
    training_acc = []
    validation_loss = []
    validation_acc = []

    # Train model for set number of epochs
    for _ in range(num_of_epochs):

        # Reset training metrics
        train_epoch_loss = 0.0
        train_epoch_acc = 0.0

        # Set model in train mode
        model.train()

        # Loop over training set in batches
        for batch in training_loader:
            
            # Unpack batch
            X = batch["Inputs"]
            Y = batch["Targets"]

            # Reset optimizer gradients
            optimizer.zero_grad()

            # Run a forward pass
            probabilities = model(X)

            # Calculate loss
            loss = loss_function(probabilities, Y)
            train_epoch_loss += loss.item()

            # Update model
            loss.backward()
            optimizer.step()

            # Calculate predictions
            predictions = probabilities >= CLASS_THR

            # Log true predictions
            train_epoch_acc += sum(predictions == Y)

        # Calculate epoch metrics
        train_epoch_loss /= len(training_loader)
        train_epoch_acc /= len(training_loader.dataset)

        # Log epoch metrics
        training_loss.append(train_epoch_loss)
        training_acc.append(train_epoch_acc)

        # Reset validation metrics
        valid_epoch_loss = 0.0
        valid_epoch_acc = 0.0

        # Set model in evaluation mode
        model.eval()

        # Loop over validation set in batches
        for batch in validation_loader:
            
            # Unpack batch
            X = batch["Inputs"]
            Y = batch["Targets"]

            # Run forward pass
            probabilities = model(X)

            # Calculate loss
            loss = loss_function(probabilities, Y)
            valid_epoch_loss += loss.item()

            # Calculate predictions
            predictions = probabilities >= CLASS_THR

            # Log true predictions
            valid_epoch_acc += sum(predictions == Y)

        # Calculate validation metrics
        valid_epoch_loss /= len(validation_loader)
        valid_epoch_acc /= len(validation_loader.dataset)

        # Log validation metrics
        validation_loss.append(valid_epoch_loss)
        validation_acc.append(valid_epoch_acc)

    # Get metrics and prediction probabilities of final model
    probabilities = []
    predictions = []
    targets = []

    # Evaluate final model
    for batch in validation_loader:
        
        # Load batch
        X = batch["Inputs"]
        Y = batch["Targets"]

        # Run forward pass
        batch_probabilities = model(X)
        batch_predictions = batch_probabilities >= CLASS_THR

        # Log probabilities, predictions and targets
        probabilities.extend(batch_probabilities.detach().numpy().flatten().tolist())
        predictions.extend(batch_predictions.detach().numpy().flatten().tolist())
        targets.extend(Y.detach().numpy().flatten().tolist())

    # Cast to numpy array
    probabilities = np.array(probabilities)
    predictions = np.array(predictions)
    targets = np.array(targets)

    # Pack all results in a dictionary
    training_parameters = {
        "train loss": training_loss,
        "train acc": training_acc,
        "validation loss": validation_loss,
        "validation acc": validation_acc,
        "Prediction probabilities": probabilities,
        "Predictions": predictions,
        "Targets": targets,
    }

    return model, training_parameters

# Treniranje različitih modela

U ovom odeljku je definisano tri modela iste arhitekture, ali sa drugačijim hiperparametrima.
Za *loss* funkciju izabrana je **binarna krosentropija**, pošto rezultati predstavljaju klasifikaciju igrača u dve klase (opstaće/neće opstati u ligi).

Od optimizacionih funkcija, eksperimentisano je sa **stohastičkim gradijentalnim spustom** i **Adamovim optimizatorem**.
Adam je pokazao konstantno bolje rezultate, pa je on korišten za sve primere.

Svi modeli su trenirani na istom datasetu, istim redosledom batcheva i na **400 epoha**.

U nastavku su definisane tri arhitekture: jedna koja je nedovoljno obučena, jedna koja je preobučena, i jedna koja je adekvatna.
Slede definicije modela, njihovi treninzi i performanse.

In [ ]:
from torch.nn import BCELoss
from torch.optim import Adam, SGD

# Initialize loss function as Binary Cross Entropy
loss_function = BCELoss()

# Define learning rate
lr = 3e-4

# Define number of epochs
num_of_epochs = 400

# Wrap training loaders into dictionary
training_objects = {
    "Loss Function": loss_function,
    "Train Dataloader": training_loader,
    "Valid Dataloader": validation_loader,
    "Epochs": num_of_epochs,
}

### Definicija, trening i evaluacija

#### Treniranje modela sa premalom arhitekturom

In [ ]:
# Define model with no hidden layers
underfit_model = NBA_Survival_Predictor([train_dataset.inputs.shape[1], 1])

# Define optimizer for model with 5 times smaller learning rate
underfit_optimizer = Adam(params=underfit_model.parameters(), lr=lr/5)

# Train and evaluate model
underfit_model, underfit_results = train_network(underfit_model, underfit_optimizer, training_objects=training_objects)

#### Treniranje modela sa prevelikom arhitekturom

In [ ]:
# Define model with 2 hidden layers
overfit_model = NBA_Survival_Predictor([train_dataset.inputs.shape[1], 20, 4, 1])

# Define optimizer for model with 3 times bigger learning rate
overfit_optimizer = Adam(params=overfit_model.parameters(), lr=lr*3)

# Train and evaluate model
overfit_model, overfit_results = train_network(overfit_model, overfit_optimizer, training_objects=training_objects)

#### Treniranje modela sa adekvatnom arhitekturom

In [ ]:
# Define model with 1 hidden layer
overfit_model = NBA_Survival_Predictor([train_dataset.inputs.shape[1], 16, 1])

# Define optimizer for model with original learning rate
overfit_optimizer = Adam(params=overfit_model.parameters(), lr=lr)

# Train and evaluate model
final_model, final_results = train_network(overfit_model, overfit_optimizer, training_objects=training_objects)

### Prikaz rezultata treninga i validacije

Prikazani su rezultati za sva tri modela.
U prvom redu su prikazane vrednosti *loss* funkcije po epohama, za validaciju i trening.
U drugom redu su prikazane tačnosti modela na treningu i validaciji po epohama.
U trećem redu grafika su postavljene konfuzione matrice na validacionom setu na kraju treninga.

In [ ]:
# Initialize 3x3 subplots
fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(20, 15))

# Initialize graph column names
title_labels = ["Underfit", "Good fit", "Overfit"]

# Loop over each column and generate graphs
for idx, results in enumerate([underfit_results, overfit_results, final_results]):

    # Plot loss
    ax[0, idx].plot(results["train loss"], label="Train")
    ax[0, idx].plot(results["validation loss"], label="Validation")
    ax[0, idx].legend()
    ax[0, idx].set_title(f"{title_labels[idx]} loss")

    # Plot accuracy
    ax[1, idx].plot(results["train acc"], label="Train")
    ax[1, idx].plot(results["validation acc"], label="Validation")
    ax[1, idx].legend()
    ax[1, idx].set_title(f"{title_labels[idx]} accuracy")

    # Plot confusion matrix
    sb.heatmap(confusion_matrix(results["Targets"], results["Predictions"]), vmin=0, vmax=len(validation_loader.dataset), annot=True, ax=ax[2, idx])
    ax[2, idx].set_title(f"{title_labels[idx]} confusion matrix")

Na graficima se može primetiti karakteristično ponašanje za sva tri modela.

Model bez skrivenih slojeva nije dovoljno kompleksan da bi efikasno vršio predviđanja.
Ovo se oslikava u bajesu modela, koji se primećuje u konstantnoj razlici između tačnosti na treningu i validaciji.

Model sa 2 skrivena sloja preobučava.
Očigledan simptom preobučavanja su funkcije *loss*-a i tačnosti na validaciji i treningu, koje posle 100. epohe počinju sve više da se udaljavaju jedna od druge.

Model sa 1 skrivenim slojem pokazuje najbolje performanse.
Funkcije treninga i validacije imaju isti oblik i polako rastu kroz epohe.
*Loss* funkcije imaju željenu eksponencijalnu raspodelu.

Zbog male veličine modela, trening za sva tri traje slično vreme, oko minut i 30 sekundi.

### Preciznost/Odziv krive najboljeg modela

Nakon odabira najboljeg modela, menjan je prag odsecanja i analizirani su preciznost i odziv modela.

In [ ]:
# Generate precision and recall curve points
prec, recall, thrs = precision_recall_curve(final_results["Targets"], final_results["Prediction probabilities"])

# Calculate f1-score
f1 = prec * recall / (prec + recall)

# Plot precision/recall/f1 curves
_, ax = plt.subplots(1, 1, figsize=(10, 5))
plt.plot(thrs, prec[:-1], label="Precision")
plt.plot(thrs, recall[:-1], label="Recall")
plt.plot(thrs, f1[:-1], label="F1")
plt.legend()
plt.show()

Funkcija preciznosti u rasponu od $0$ do $0.5$ raste sporije nego što funkcija odziva pada u istom segmentu.
Kako je cilj modela da predvidi koje igrače treba zadržati u klubu, poželjan je visok odziv.
Visok odziv odgovora pošto su igrači u prvim godinama karijere relativno jeftini za zadržavanje, a uvek ih je moguće menjati kasnije.

Sa druge strane, preciznost ne sme pasti značajno, kako klub ne bi plaćao igrače koji neće igrati na duže staze.

Kompromis između ova dva cilja možemo naći na pragu odsecanja od $0.45$.\
Ovim smanjenjem praga će tačnost pasti, ali če odziv skočiti. Ova promena obezbeđuje da model zadržava više igrača koji će igrati u NBA.

In [ ]:
# Define final threshold
final_threshold = 0.45

# Get index of threshold with value of final threshold
thr_idx = sum(thrs <= final_threshold)

# Get best model precision and recall
final_precision = prec[thr_idx]
final_recall = recall[thr_idx]
final_f1 = f1[thr_idx]

# Calculate accuracy with final threshold
final_accuracy = sum(final_results["Targets"] == (final_results["Prediction probabilities"] > final_threshold))
final_accuracy /= len(final_results["Targets"])

# Print final results
print("Statistika finalnog modela:")
print("---------------------------")
print(f"\tTačnost:    {final_accuracy * 100:.1f}%")
print(f"\tPreciznost: {final_precision * 100:.1f}%")
print(f"\tOdziv:      {final_recall * 100:.1f}%")
print(f"\tF1 skor:    {final_accuracy * 100:.1f}%")

# Automatsko traženje hyperparametara

Nakon ručnog odabira hiperparametara, implementirana je automatska pretraga.
U ovu svrhu upotrebljena je biblioteka ```optuna```, koju je koristiti sa već implementiranim modelom bez izmena.

Biblioteka radi generisanjem *study* sesije, kojoj treba zadati fukciju koju treba optimizovati.
Ova funkcija se definiše odvojeno, i u njoj je moguće generisati hiperparametre iz proizvoljne raspodele i nakon toga trenirati tako definisan model.

*Study* sesiji se takođe prosleđuje i sampler.
Cilj samplera je blago menjanje funkcija raspodela za odabir hiperparametara u zavisnosti od njihove performanse tokom sesije.

In [ ]:
import optuna

### Funkcija cilja

U funkciji cilja su birani *learning rate*, broj skrivenih slojeva, kao i veličina svakog od skrivenih slojeva.
*Learning rate* je biran iz raspodele koja je uniformna na logaritamskoj skali u rasponu od $10^{-6}$ do $10^{-4}$.
Broj skrivenih slojeva je biran nasumično između 1, 2, 3 i 4, a svaki layer ima između 3 i 20 neurona.

Svaki model je treniran na $500$ epoha.

Rezultat funkcije cilja je maksimalna tačnost na validaciji modela.

In [ ]:
def objective_function(trial: optuna.trial.Trial) -> float:
    """Objective function for NBA_Survival_Predictor training.

    Args:
        trial (optuna.trial.Trial): Optuna trial object.

    Returns:
        float: Maximal validation accuracy of the model.
    """
    # Pick learning rate from loguniform distribution
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)
    
    # Pick number of layers
    num_of_layers = trial.suggest_int("num_of_layers", 1, 4)

    # Initialize layer size list with input layer size
    layer_sizes = [train_dataset.inputs.shape[1]]

    # For each layer pick its size
    for idx in range(num_of_layers):
        layer_sizes.append(trial.suggest_int(f"layer_{idx}", 3, 20))
    
    # Add output layer size to layer sizes list
    layer_sizes.append(1)

    # Init model
    model = NBA_Survival_Predictor(layer_sizes)
    
    # Init optimizer
    optimizer = Adam(model.parameters(), lr=lr)

    # Set number of epochs
    training_objects["Epochs"] = 500

    # Train and evaluate model
    _, results = train_network(model, optimizer, training_objects)

    # Return best performance
    return max(results["validation acc"])

### Traženje najboljih hiperparametara

U nastavku je kreiran *study* koji pokušava da maksimizuje prethodnu funkciju cilja.
Definisan je maksimum od $100$ pokušaja za maksimizaciju funkcije.

In [ ]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective_function, n_trials=100)

# Redukcija dimenzionalnosti

Kao poslednji pokušaj za unapređenje modela, implementirana je redukcija dimenzionalnosti *dataset*-a.\
Implementirana je **LDA** redukcija i set atributa je redukovan na $4$.\
Redukcija je implementirana na normalizovanim podacima.

Nad ovakvim podacima treniran je perceptron, zbog malog broja atributa.
*Learning rate* je menjan u ```Optuna``` *study*-u.

### LDA redukcija
Kod ispod generiše kovariacionu matricu atributa i računa njihove sopstvene vektore.\
Od $4$ vektora, koji odgovaraju atributima sa najvećim sopstvenim vrednostim, se formira matrica kojom ce biti transformisan *dataset*. 

In [ ]:
# Calculate covariance matrix of attributes, without targets
cov_mat = norm_train_df.corr().to_numpy()[:-1, :-1]

# Calculate eigenvalues
eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)

# Take vector subset
num_of_components = 4
eigv_subset = eigen_vectors[:, -num_of_components:]

Transformišemo trening i test podatke dobijenom matricom.

In [ ]:
# Extract numpy matrix of training attributes
X = norm_train_df.to_numpy()[:, :-1]

# Calculate reduced set of attributes
X_reduced = np.dot(eigv_subset.T, X.T).T

# Initialize datafreme with reduced attributes
reduced_train_df = pd.DataFrame(X_reduced, columns=[f"feature {idx}" for idx in range(num_of_components)])

# Add target values to the dataframe
reduced_train_df["target"] = norm_train_df["target"]

# Print results
print("Redukovani trening dataframe:")
reduced_train_df

In [ ]:
# Extract numpy matrix of validation attributes
X = norm_valid_df.to_numpy()[:, :-1]

# Calculate reduced set of attributes
X_reduced = np.dot(eigv_subset.T, X.T).T

# Initialize datafreme with reduced attributes
reduced_valid_df = pd.DataFrame(X_reduced, columns=[f"feature {idx}" for idx in range(num_of_components)])

# Add target values to the dataframe
reduced_valid_df["target"] = norm_valid_df["target"]

# Print results
print("Redukovani validacioni dataframe:")
reduced_valid_df

### Generisanje *dataloader*-a

In [ ]:
reduced_train_dataset = NBA_Dataset(reduced_train_df)

train_parameters = {
    "batch_size": 64,
    "shuffle": True,
    "num_workers": 1,
}

reduced_training_loader = DataLoader(reduced_train_dataset, **train_parameters)

# Validation dataset
reduced_valid_dataset = NBA_Dataset(reduced_valid_df)

valid_parameters = {
    "batch_size": 64,
    "shuffle": False,
    "num_workers": 1,
}

reduced_validation_loader = DataLoader(reduced_valid_dataset, **valid_parameters)

Definisanje trening objekata

In [ ]:
# Initialize training objects dictionary
training_objects = {
    "Loss Function": loss_function,
    "Train Dataloader": reduced_training_loader,
    "Valid Dataloader": reduced_validation_loader,
    "Epochs": num_of_epochs,
}

### Treniranje modela

Pošto je trenirani model perceptron, jedini hiperparametar koji poseduje je *learning rate*.
Implementirana je nova funkcija cilja koja pretražuje optimalan *learning rate*.

In [ ]:
def objective_function_reduced(trial: optuna.trial.Trial) -> float:
    """Objective function for NBA_Survival_Predictor training, with reduced feature space.

    Args:
        trial (optuna.trial.Trial): Optuna trial object.

    Returns:
        float: Maximal validation accuracy of the model.
    """
    # Pick learning rate from loguniform distribution
    lr = trial.suggest_loguniform("learning_rate", 1e-6, 1e-4)

    layer_sizes = [4, 1]

    # Init model
    model = NBA_Survival_Predictor(layer_sizes)
    
    # Init optimizer
    optimizer = Adam(model.parameters(), lr=lr)

    # Set number of epochs
    training_objects["Epochs"] = 300

    # Train and evaluate model
    _, results = train_network(model, optimizer, training_objects)

    # Return best performance
    return max(results["validation acc"])

In [ ]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective_function_reduced, n_trials=5)